# Grosser Aletschgletscher Visual Assessment 

In [2]:
import io
import os
import time
import numpy as np
import PIL.Image
import requests

from datetime import datetime, timedelta
from PIL import Image

## Data Collection and Storage

In [6]:
TILE_COL = 167
TILE_ROW = 38

BASE_URL = "https://gibs-b.earthdata.nasa.gov/wmts/epsg4326/best/wmts.cgi?TIME=[TIME]&layer=MODIS_Terra_CorrectedReflectance_TrueColor&style=default&tilematrixset=250m&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fjpeg&TileMatrix=8&TileCol=[TILE_COL]&TileRow=[TILE_ROW]"

def get_data_for_date(date: datetime):
    url = BASE_URL.replace("[TIME]", date.strftime("%Y-%m-%d"))
    url = url.replace("[TILE_COL]", str(TILE_COL))
    url = url.replace("[TILE_ROW]", str(TILE_ROW))
    response = requests.get(url, timeout=5)
    return response.content

def get_image_for_date(date: datetime):
    data = get_data_for_date(date)
    image = PIL.Image.open(io.BytesIO(data))
    return np.array(image)

start_date = datetime(2004, 1, 1)
end_date = datetime(2020, 1, 1)

def retry(func, wait=1):
    if wait > 16:
        raise Exception("Max retries exceeded")
    while True:
        try:
            return func()
        except Exception as e:
            time.sleep(wait)
            return retry(func, wait * 2)

# Create data directory
os.makedirs("satellite", exist_ok=True)
os.makedirs(f"satellite/{TILE_COL}_{TILE_ROW}", exist_ok=True)

# check if images are already downloaded (if so, use last date)
if os.listdir(f"satellite/{TILE_COL}_{TILE_ROW}"):
    # Get last date
    start_date = \
    sorted([datetime.strptime(file.split(".")[0], "%Y-%m-%d") for file in os.listdir(f"satellite/{TILE_COL}_{TILE_ROW}")])[
        -1]

for date in [start_date + timedelta(days=i) for i in range((end_date - start_date).days)]:
    try:
        image = retry(lambda: get_image_for_date(date))
        PIL.Image.fromarray(image).save(f"satellite/{TILE_COL}_{TILE_ROW}/{date.strftime('%Y-%m-%d')}.jpg")
        print(f"Saved {date.strftime('%Y-%m-%d')}")
    except Exception as e:
        print(f"Error {date.strftime('%Y-%m-%d')}: {e}")
        continue

Saved 2019-12-31


## Data Analysis

In [ ]:
"""
Image Analysis

For each image in the satellite folder (date (YYYY-MM-DD).jpg), compute the brightness of the image.
""" 
os.makedirs("data/results", exist_ok=True)

for folder in os.listdir("satellite"):
    data = []
    # open csv
    with open(f"results/{folder}.csv", "w") as f:
        f.write("date,brightness,white\n")
        prev_imag = None
        for file in sorted(os.listdir(f"satellite/{folder}")):
            date = file.split(".")[0]
            image = Image.open(f"satellite/{folder}/{file}")
            total_brightness = np.mean(image)

            if total_brightness < 5 or total_brightness > 245:
                print(f"Skipping {date} due to brightness {total_brightness}")
                if prev_imag is not None:
                    image = prev_imag
                else:
                    continue

            prev_imag = image

            image = np.array(image)
            # Cut image to zone of interest
            image = image[300:400, 40:140]
            image = np.array(image)
            brightness = np.mean(image)

            # Compute the percentage of pixels that are "white" (each channel > 230)
            white = np.mean((image > 230).all(axis=2))

            f.write(f"{date},{brightness},{white}\n")
            data.append((date, brightness, white))

        # Remove if brightness is  < 5 or > 245
        data = [(date, white) for date, brightness, white in data if 5 < brightness < 245]

        # plot
        import matplotlib.pyplot as plt

        # Apply moving average
        def ma(data, window=25):
            # Apply moving average using median
            return np.convolve(data, np.ones(window) / window, mode="valid")

        dates, brightness = zip(*data)
        brightness_ma = ma(brightness)
        dates = dates[len(dates) - len(brightness_ma):]
        plt.plot(dates, brightness_ma)
        plt.title(folder)
        # x label is only month
        plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter("%Y-%m"))
        plt.xlabel("Date")
        plt.ylabel("Brightness")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        # Now sum per year to show evolution of whitness
        from collections import defaultdict

        data_per_year = defaultdict(list)

        for date, white in data:
            year = date.split("-")[0]
            data_per_year[year].append(white)

        years = []
        white = []
        for year, values in data_per_year.items():
            years.append(year)
            white.append(np.mean(values))

        plt.plot(years, white)
        plt.title(folder)
        plt.xlabel("Year")
        plt.ylabel("White")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()